In [1]:
using Statistics, CSV, DataFrames, Shuffle

In [123]:
# using Pkg
# Pkg.add("Shuffle")

In [2]:
include("utils.jl")
include("synthdid.jl")
include("solver.jl")
include("data.jl")
include("vcov.jl")

sum_normalize (generic function with 1 method)

In [3]:
# include("synthdid.jl")

In [4]:
# function sum_normalize(x)
#     if sum(x) != 0
#         x/sum(x)
#     else
#         repeat([1/length(x)], length(x))
#     end
# end

In [3]:
setup = panel_matrices(data("california_prop99"));

algo = synthdid_estimate(setup.Y, setup.N0, setup.T0, omega_intercept=true);
estimate = algo;
replications = 5;

In [12]:
Y = algo.setup["Y"]
N0 = algo.setup["N0"]
T0 = algo.setup["T0"]
X::Array=Array{Any,3}(undef, size(Y, 1), size(Y, 2), 0);
# X = zeros([size(Y, 1), size(Y, 2), 0])
noise_level::Float64=std(diff(Y[1:N0, 1:T0], dims=2))
eta_omega::Float64=((size(Y, 1) - N0) * (size(Y, 2) - T0))^(1 / 4)
eta_lambda::Float64=1e-6
zeta_omega::Float64=eta_omega * noise_level
zeta_lambda::Float64=eta_lambda * noise_level
omega_intercept::Bool=true
lambda_intercept::Bool=true
weights=Dict("omega" => nothing, "lambda" => nothing, "vals" => [1, 2, 3.0])
update_omega::Bool=isnothing(weights["omega"])
update_lambda::Bool=isnothing(weights["lambda"])
min_decrease::Float64=1e-5 * noise_level
max_iter::Int=10000
sparsify::Function=sparsify_function
max_iter_pre_sparsify::Int=100

100

In [14]:
# if (!(size(Y)[1] > N0) && !(size(Y)[2] > T0) && !(length(size(X)) == 2 || length(size(X)) == 3) & !(size(X)[1:2] == size(Y)) && !(isa(weights, Dict)) 
#     && !((isnothing(weights["lambda"])) || (length(weights["lambda"]) == T0)) && !((isnothing(weights["omega"])) || (length(weights["omega"]) == N0))
#     && !(!(isnothing(weights["lambda"])) || (update_lambda)) && !((!isnothing(weights["omega"]) || (update_omega))))

#     error("error at !(size(Y)[1] > N0) || !(size(Y)[2] > T0) || ... in synthdid_estimate function")
#   else
#     "continue"
# end

In [5]:
function placebo_se(estimate, replications)
    setup = estimate.setup
    opts = estimate.opts
    weights = estimate.weight
    N1 = size(setup["Y"])[1] - setup["N0"]
    if setup["N0"] <= N1
        error("must have more controls than treated units to use the placebo se")
    end
    function theta(ind)
        N0 = length(ind)-N1
        weights1 = copy(weights)
        weights_boot = weights1
        weights_boot["omega"] = sum_normalize(weights["omega"][ind[1:N0]]);
        synthdid_estimate(setup["Y"][ind,:], N0, setup["T0"],  X=setup["X"][ind, :,:], weights=weights_boot,
                            zeta_omega = opts["zeta_omega"], zeta_lambda = opts["zeta_lambda"], omega_intercept = opts["omega_intercept"],
                            lambda_intercept = opts["lambda_intercept"], update_omega = opts["update_omega"], update_lambda = opts["update_lambda"],
                            min_decrease = opts["min_decrease"], max_iter = opts["max_iter"])
    end
    
    a = 0
    for i in 1:replications
        if i == 1
            a = theta(shuffle(1:setup["N0"])).estimate
        else
            a = vcat(a,theta(shuffle(1:setup["N0"])).estimate)
        end
    end
    sqrt((replications-1)/replications) * std(a)
end

In [6]:
placebo_se(algo, 5)

5.487047692950628

In [715]:
function bootstrap_sample(estimate, replications)
    setup = estimate.setup
    opts = estimate.opts
    weights = estimate.weight
    if setup["N0"] == size(setup["Y"])[1] - 1
        return NaN
    end
    function theta(ind)
        if all(ind .<= setup["N0"]) || all(ind .> setup["N0"])
            NaN
        else
            weights1 = copy(weights)
            weights_boot = weights1
            weights_boot["omega"] = sum_normalize(weights["omega"][sort(ind[ind .<= setup["N0"]])])
            synthdid_estimate(setup["Y"][sort(ind),:], sum(ind .<= setup["N0"]), setup["T0"],  X=setup["X"][sort(ind), :,:], weights=weights_boot,
                                zeta_omega = opts["zeta_omega"], zeta_lambda = opts["zeta_lambda"], omega_intercept = opts["omega_intercept"],
                                lambda_intercept = opts["lambda_intercept"], update_omega = opts["update_omega"], update_lambda = opts["update_lambda"],
                                min_decrease = opts["min_decrease"], max_iter = opts["max_iter"])
        end
    end

    bootstrap_estimates = repeat([NaN], replications)
    count = 0
    while count < replications
        a = theta(sample(1:size(setup["Y"])[1], size(setup["Y"])[1], replace=true))
        if typeof(a) == synthdid_est1
            bootstrap_estimates[count+1] = a.estimate
            count = count+1
        end
    end
    bootstrap_estimates
end

bootstrap_sample (generic function with 2 methods)

In [9]:
bootstrap_sample(estimate, replications)

NaN

In [552]:
function bootstrap_se(estimate, replications)
    sqrt((replications-1)/replications) * std(bootstrap_sample(estimate, replications))
end

bootstrap_se (generic function with 1 method)

In [6]:
bootstrap_se(estimate, 5)

NaN

In [719]:
setup = panel_matrices(data("california_prop99"));

algo = synthdid_estimate(setup.Y, setup.N0, setup.T0, omega_intercept=true);
estimate = algo;
replications = 5;

In [12]:
function jackknife_se(estimate; weights = estimate.weight)
    setup = estimate.setup
    opts = estimate.opts
    weights = estimate.weight
    if !isnothing(weights)
        opts["update_omega"] = opts["update_lambda"] = false
    end
    if setup["N0"] == size(setup["Y"])[1] - 1 || (!isnothing(weights) && sum(weights["omega"] != 0) == 1) 
        return NaN
    end
    
    function jackknife(x)
        n = length(x)
        u = repeat([0.0], n)
        function theta(ind)
            weights1 = copy(weights)
            weights_jk = weights1
            if !isnothing(weights)
                weights_jk["omega"] = sum_normalize(weights["omega"][ind[ind .<= setup["N0"]]])
            end
            synthdid_estimate(setup["Y"][ind,:], sum(ind .<= setup["N0"]), setup["T0"],  X=setup["X"][ind, :,:], weights=weights_jk,
                                        zeta_omega = opts["zeta_omega"], zeta_lambda = opts["zeta_lambda"], omega_intercept = opts["omega_intercept"],
                                        lambda_intercept = opts["lambda_intercept"], update_omega = opts["update_omega"], update_lambda = opts["update_lambda"],
                                        min_decrease = opts["min_decrease"], max_iter = opts["max_iter"])
        end
        for i in 1:n 
          u[i] = theta(x[Not(i)]).estimate
        end
        jack_se = sqrt(((n - 1) / n) * (n - 1) * var(u))
      
        jack_se
    end
    jackknife(1:size(setup["Y"])[1])
end

jackknife_se (generic function with 1 method)

In [13]:
jackknife_se(estimate)

NaN

In [666]:
function vcov_synthdid_estimate(object; method = "bootstrap", replications = 200)
    if method == "bootstrap"
        se = bootstrap_se(object, replications)
    elseif method == "jackknife"
        se = jackknife_se(object)
    elseif method == "placebo"
        se = placebo_se(object, replications)
    end
    se^2
end

vcov_synthdid_estimate (generic function with 1 method)

In [24]:
vcov_synthdid_estimate(algo, method = "jackknife")

NaN

In [22]:
vcov_synthdid_estimate(algo, method = "bootstrap")

NaN

In [18]:
vcov_synthdid_estimate(algo, method = "placebo")

78.92802558115682